This notebook takes the raw tracks for each observation and finds all crossing tracks and extracts all nessisary assosiated information for population estimates including things like bat size, frame darkness etc. All this information is saved in one dictionary for each observation which can be used in other notebooks for further processing.

In [ ]:
import glob
import os

import cv2
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import utm

from bat_functions import mark_bats_on_image, draw_tracks_on_frame
from bat_functions import get_bat_accumulation, piecewise_linear
from bat_functions import threshold_short_tracks, calculate_height
from bat_functions import calculate_bat_multiplier, combined_bat_multiplier
from bat_functions import measure_crossing_bats
from CountLine import CountLine

In [ ]:
shift = 0 # loss on each side from not padding during detection

HFOV = 85.8 # degrees
HCONST = 1454.9 # pixels
WCONST = 1453.7 # pixels
FRAME_WIDTH = 2704 - (2 * shift)
WINGSPAN = .8 # meters, max extent while flying 
# Path to a frame extracted from the videos (to get the height and width of the videos)
image_file = '.../kasanka-bats/processed/deep-learning/19Nov/BBC/example-frames/19Nov_BBC_obs-ind_40500.jpg'
frame_height = plt.imread(image_file).shape[0]

In [ ]:
root_folder = ".../kasanka-bats/processed/deep-learning"

## Take all tracks found and filter out all that don't cross the count line
Save these crossing tracks seprately so this only has to be run once

In [ ]:
# Get path to the folders for each observation date
# (Each of these contains folders for each camera which contain 'raw_tracks.npy' etc)
day_folders = day_folders = sorted(
    glob.glob('.../kasanka-bats/processed/deep-learning/*Nov')
)

observations = {}
for day_folder in day_folders:
    print(day_folder)
    date = os.path.basename(day_folder) 
    track_files = sorted(
        glob.glob(os.path.join(day_folder, '*/raw_tracks.npy'))
    )
    observations[date] = {}
    for track_file in track_files:
        camera_folder = os.path.dirname(track_file)
        # New file path for the crossing tracks that will be saved
        crossing_track_file = os.path.join(camera_folder, 'crossing_tracks.npy')
        if os.path.exists(crossing_track_file):
            # If the crossing tracks from this camera have already been saved don't
            # refind them
            continue
        try:
            raw_track_list = np.load(track_file, allow_pickle=True)
            # Get rid of tracks less than two points long
            tracks_list = threshold_short_tracks(raw_track_list, min_length_threshold=2)
            # Get list of tracks that cross the mid line
            crossing_tracks_list = measure_crossing_bats(tracks_list, 
                                                         frame_height=frame_height)
            np.save(crossing_track_file, 
                    np.array(crossing_tracks_list, dtype=object)
                   )
        except:
            print(f"Filtering and saving of crossing tracks for {date}"
                  f" and camera {os.path.basename(camera_folder)}failed.")

### For each camera observation on each day, complile all nessisary information about each crossing bat to estimate the total population

In [ ]:
day_folders = sorted(
    glob.glob('.../kasanka-bats/processed/deep-learning/*Nov')
)

observations = {}
for day_folder in day_folders:
    print(day_folder)

    date = os.path.basename(day_folder)
    track_files = sorted(glob.glob(os.path.join(day_folder, '*/crossing_tracks.npy')))
    observations[date] = {}
    
    for track_file in track_files: 
        camera = track_file.split('/')[-2]
        obs = {'date': date,
               'camera': camera,
               }
        crossing_tracks_list = np.load(track_file, allow_pickle=True)
        # Load the frame darkness values as calculated in get-observation-frame-darkness.ipynb
        darkness_means = np.load(os.path.join(day_folder, camera, 'blue-means.npy'))
        passing_bat_frame = []
        passing_bat_size = []
        passing_bat_track_ind = []
        passing_bat_direction = []
        passing_bat_track_id = []
        passing_bat_darkness = []
        passing_track_length = []
        for track_ind, track in enumerate(crossing_tracks_list):
            if track['crossed'] > 0:
                passing_bat_frame.append(track['crossed'])
                passing_bat_size.append(track['mean_wing'])
                passing_bat_track_id.append(track_ind)
                passing_bat_direction.append(1)
                passing_bat_darkness.append(darkness_means[track['crossed']])
                passing_track_length.append(len(track['track']))

            elif track['crossed'] < 0:
                passing_bat_frame.append(track['crossed'])
                passing_bat_size.append(track['mean_wing'])
                passing_bat_track_id.append(track_ind)
                passing_bat_direction.append(-1)
                passing_bat_darkness.append(darkness_means[-track['crossed']])
                passing_track_length.append(len(track['track']))

        obs['frames'] = np.array(passing_bat_frame)
        obs['mean_wing'] = np.array(passing_bat_size)
        obs['ids'] = np.array(passing_bat_track_id)
        obs['direction'] = np.array(passing_bat_direction)
        obs['darkness'] = np.array(passing_bat_darkness)
        obs['track_length'] = np.array(passing_track_length)
        observations[date][camera] = obs
        
# Save the track crossing information for each observation (camera, date)
observation_root = os.path.join(root_folder, 'observations')
for date, day_obs in observations.items():
    day_folder = os.path.join(observation_root, date)
    os.makedirs(day_folder, exist_ok=True)
    for cam_ind, (cam_name, obs) in enumerate(day_obs.items()):
        obs_name = f'{date}-observation-{cam_name}.npy'
        obs_file = os.path.join(day_folder, obs_name)
        np.save(obs_file, obs)